In [0]:
username = 'scook'
from IPython.display import display, HTML, clear_output
try:
    %reload_ext autotime
except:
    %pip install -U ipython-autotime ipywidgets codetiming openpyxl numpy pandas geopandas pgeocode flaml[automl] git+https://github.com/AnotherSamWilson/miceforest.git
    dbutils.library.restartPython()
    clear_output()
    dbutils.notebook.exit('Rerun to use newly installed/updated packages')

import pathlib, shutil, pickle, warnings, requests, dataclasses, codetiming, numpy as np, pandas as pd, geopandas as gpd, pgeocode, flaml as fl, miceforest as mf
clear_output()
catalog = 'dev.bronze.'
root = pathlib.Path(f'/Workspace/Users/{username}@tarleton.edu/admitted_matriculation_predictor_2025/')
data = root/'data'
flags_raw = pathlib.Path('/Volumes/aiml/scook/scook_files/admitted_flags_raw')
flags_prc = pathlib.Path('/Volumes/aiml/flags/flags_volume/')

############ annoying warnings to suppress ############
for w in [
    "Could not infer format, so each element will be parsed individually, falling back to `dateutil`",
#     "Engine has switched to 'python' because numexpr does not support extension array dtypes",
    # "The default of observed=False is deprecated and will be changed to True in a future version of pandas",
    # "errors='ignore' is deprecated"
    # "The behavior of DataFrame concatenation with empty or all-NA entries is deprecated",
    # "The behavior of array concatenation with empty entries is deprecated",
    # "DataFrame is highly fragmented",
    # "DataFrameGroupBy.apply operated on the grouping columns",
    ]:
    warnings.filterwarnings(action='ignore', message=f".*{w}.*")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
##########################################
############ helper functions ############
##########################################
tab = '    '
divider = '\n##############################################################################################################\n'

def setmeth(cls, fcn):
    """monkey-patch new method into a mutable class (fails for immutable class)"""
    setattr(cls, fcn.__name__, fcn)

def listify(*args):
    """ensure it is a list"""
    if len(args)==1:
        if args[0] is None or args[0] is np.nan or args[0] is pd.NA:
            return list()
        elif isinstance(args[0], str):
            return [args[0]]
    try:
        return list(*args)
    except Exception as e:
        return list(args)

def setify(*args):
    """ensure it is a set"""
    return set(listify(*args))

def unique(*args):
    """get unique items maintaining order"""
    return listify(dict.fromkeys(listify(*args)))

def difference(A, B):
    return unique([x for x in listify(A) if x not in listify(B)])

def rjust(x, width, fillchar=' '):
    return str(x).rjust(width,str(fillchar))

def ljust(x, width, fillchar=' '):
    return str(x).ljust(width,str(fillchar))

def join(lst, sep='\n,', pre='', post=''):
    """flexible way to join list of strings into a single string"""
    return f"{pre}{str(sep).join(map(str,listify(lst)))}{post}"

def alias(dct):
    """convert dict of original column name:new column name into list"""
    return [f'{k} as {v}' for k,v in dct.items()]

def indent(x, lev=1):
    return x.replace('\n','\n'+tab*lev) if lev>0 else x

def subqry(qry, lev=1):
    """make qry into subquery"""
    qry = '\n' + qry.strip()
    qry = '(' + qry + '\n)' if 'select' in qry else qry
    return indent(qry, lev)

def run(qry, show=False, sample='10 rows', seed=42):
    """run qry and return dataframe"""
    L = qry.split(' ')
    if len(L) == 1:
        qry = f'select * from {catalog}{L[0]}'
        if sample is not None:
            qry += f' tablesample ({sample}) repeatable ({seed})'
    if show:
        print(qry)
    return spark.sql(qry).toPandas().prep().sort_index()

def rm(path, root=False):
    path = pathlib.Path(path)
    if path.is_file():
        path.unlink()
    elif path.is_dir():
        if root:
            shutil.rmtree(path)
        else:
            for p in path.iterdir():
                rm(p, True)
    return path

def get_desc(code):
    for nm in code.split('_'):
        if len(nm) == 4:
            break
    return f'{code} as {nm}_code, (select stv{nm}_desc from {catalog}saturnstv{nm} where {code} = stv{nm}_code limit 1) as {nm}_desc'

def coalesce(x, y=False):
    return f'coalesce({x}, {y}) as {x}'

############ pandas functions ############
pd.options.display.max_columns = None
def disp(df, rows=4, head=True, sort=False):
    """convenient display method"""
    df = df.sort_index(axis=1) if sort else df
    # missing = df.isnull().sum().to_frame().T
    # X = pd.concat([df.dtypes.to_frame().T, missing, (missing/df.shape[0]*100).round(2), df.head(rows) if head else df.tails(rows)])
    X = df if rows < 0 else df.head(rows) if head else df.tails(rows)
    display(HTML(X.to_html()))
    print(df.shape)

def to_numeric(df, downcast='integer', errors='ignore', category=False, **kwargs):
    """convert to numeric dtypes if possible"""
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return (
            df
            .apply(lambda s: s.astype('string').str.lower().str.strip() if s.dtype in ['object','string'] else s)  # prep strings
            .apply(lambda s: s if pd.api.types.is_datetime64_any_dtype(s) else pd.to_numeric(s, downcast=downcast, errors=errors, **kwargs))  # convert to numeric if possible
            .convert_dtypes()  # convert to new nullable dtypes
            .apply(lambda s: s.astype('Int64') if pd.api.types.is_integer_dtype(s) else s.astype('category') if s.dtype=='string' and category else s)
        )

def prep(df, **kwargs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        h = lambda x: x.to_numeric(**kwargs).rename(columns=lambda s: s.lower().strip().replace(' ','_').replace('-','_') if isinstance(s, str) else s)
        idx = h(df[[]].reset_index())  # drop columns, reset_index to move index to columns, then apply g
        return h(df).reset_index(drop=True).set_index(pd.MultiIndex.from_frame(idx))  # set idx back to df's index

def groupb(df, by=None, sort=False, observed=False, dropna=False, **kwargs):
    return df.groupby(by=by, sort=sort, observed=observed, dropna=dropna, **kwargs)

def get_duplicates(df, subset='pidm', quit=True, rows=10):
    mask = df.groupby(subset).transform('size') > 1
    if mask.any():
        df[mask].disp(rows)
        if quit:
            raise Exception(f'{mask.sum()} duplicates detected')
    return df[mask]

def get_missing(df, rows=-1):
    miss = df.isnull().mean()*100
    if miss.any():
        miss[miss>0].sort_values(ascending=False).round(1).disp(rows)
    return miss

# def inser(df, column, value, loc=0):
#     df.insert(loc, column, value)
#     return df

def wrap(fcn):
    """Make new methods work for Series and DataFrames"""
    def wrapper(X, *args, **kwargs):
        df = fcn(pd.DataFrame(X), *args, **kwargs)
        return None if df is None else df.squeeze() if isinstance(X, pd.Series) else df  # squeeze to series if input was series
    wrapper.__name__ = fcn.__name__
    return wrapper

for fcn in [
    disp,
    to_numeric,
    prep,
    # inser,
    get_duplicates,
    get_missing,
    groupb,
    ]:
    """monkey-patch my helpers into Pandas Series & DataFrame classees so we can use df.method syntax"""
    setmeth(pd.DataFrame, fcn)
    setmeth(pd.Series, wrap(fcn))
    # setattr(pd.DataFrame, f.__name__, f)
    # setattr(pd.Series, f.__name__, wrap(f))
#########################################
################## AMP ##################
#########################################
@dataclasses.dataclass
class Term():
    term_code: int = 202408
    cycle_day: int = None
    cycle_date: str = None
    overwrite: set = None
    seed: int = 42

    #Allows self['attr'] and self.attr syntax
    def __contains__(self, key):
        return hasattr(self, key)
    def __getitem__(self, key):
        return getattr(self, key)
    def __setitem__(self, key, val):
        setattr(self, key, val)
    def __delitem__(self, key):
        if key in self:
            delattr(self, key)


    def __post_init__(self):
        self.term_code = int(self.term_code)
        self.cycle_day, self.cycle_date, self.stable_date, self.stem, self.term_desc, self.year = self.get_cycle(self.term_code, self.cycle_day, self.cycle_date)
        self.overwrite = setify(self.overwrite)
        # Because these take about 1 hour each, we force user to manually delete parquet files to avoid accidental deletion requring lengthy re-creation
        self.overwrite.discard('drivetimes_s')
        self.overwrite.discard('drivetimes_m')
        self.overwrite.discard('drivetimes_w')
        self.overwrite.discard('drivetimes_r')
        self.overwrite.discard('drivetimes_l')


    def get(self, fcn, dst, prereq=[], *, divide=True, read=True, suffix='.parquet', **kwargs):
        # nm = str(dst)
        if '/' in str(dst):
            dst = pathlib.Path(dst).with_suffix(suffix)
            nm = dst.stem
        else:
            nm = str(dst).split('_')[0]
            dst = data/f'{nm}/{self.term_code}/{dst}_{self.stem}{suffix}'

        if nm in self.overwrite:
            del self[nm]
            dst.unlink(missing_ok=True)
            self.overwrite.remove(nm)

        new = False
        if not nm in self:
            if not dst.exists():
                new = True
                for f in listify(prereq):
                    f()
                print(f'creating {dst.name}: ', end='')
                with codetiming.Timer():
                    rslt = fcn(**kwargs)
                    dst.parent.mkdir(parents=True, exist_ok=True)
                    if suffix == '.parquet':
                        pd.DataFrame(rslt).prep().to_parquet(dst)  # forced to wrap with explicit pd.DataFrame to due strange error under pandas 2.2.3 "Object of type PlanMetrics is not JSON serializable" with to_parquet
                    elif suffix == 'csv':
                        pd.DataFrame(rslt).prep().to_csv(dst)
                    else:
                        with open(dst, 'wb') as f:
                            pickle.dump(rslt, f, pickle.HIGHEST_PROTOCOL)
                if divide:
                    print(divider)
            if read:
                if suffix == '.parquet':
                    self[nm] = pd.read_parquet(dst)
                elif suffix == 'csv':
                    self[nm] = pd.read_csv(dst)
                else:
                    with open(dst, 'rb') as f:
                        self[nm] = pickle.load(f)
            else:
                self[nm] = None
        return self[nm], new
##################################################
################# get drivetimes #################
##################################################
    def get_zips(self, show=False):
        def fcn():
            df = (
                pgeocode.Nominatim('us')._data  # get all zips
                .prep()
                .rename(columns={'postal_code':'zip'})
                .query("state_code.notnull() & state_code not in [None,'mh']")
            )
            return df
        df, new = self.get(fcn, root/'geo/zips')
        return df


    def get_states(self, show=False):
        return set(self.get_zips()['state_code'])


    def get_drivetimes(self, show=False):
        def fcn():
            from pgeocode import Nominatim
            from sklearn.metrics.pairwise import haversine_distances
            print()
            campus_coords = {
                's': '-98.215784,32.216217',
                'm': '-97.432975,32.582436',
                'w': '-97.172176,31.587908',
                'r': '-96.467920,30.642055',
                'l': '-96.983211,32.462267',
                }
            url = "https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_zcta520_500k.zip"
            gdf = gpd.read_file(url).prep().set_index('zcta5ce20')  # get all ZCTA https://www.census.gov/programs-surveys/geography/guidance/geo-areas/zctas.html
            pts = gdf.sample_points(size=10, method="uniform").explode().apply(lambda g: f"{g.x},{g.y}")  # sample 10 random points in each ZCTA
            M = []
            for k, v in campus_coords.items():
                def fcn1():
                    print()
                    L = []
                    i = 0
                    di = 200
                    I = pts.shape[0]
                    while i < I:
                        u = join([v, *pts.iloc[i:i+di]],';')
                        url = f"http://router.project-osrm.org/table/v1/driving/{u}?sources={0}&annotations=duration,distance&fallback_speed=1&fallback_coordinate=snapped"
                        response = requests.get(url).json()
                        L.append(np.squeeze(response['durations'])[1:]/60)
                        i += di
                        print(k,i,round(i/I*100))
                    df = pts.to_frame()[[]]
                    df[k] = np.concatenate(L)
                    return df
                df, new = self.get(fcn1, root/f'geo/drivetimes_{k}')
                M.append(df)
            D = pd.concat(M, axis=1).groupb(level=0).min().stack().reset_index().set_axis(['zip','camp_code','drivetime'], axis=1)

            # There are a few USPS zips without equivalent ZCTA, so we assign them drivetimes for the nearest
            Z = self.get_zips().merge(D.query("camp_code=='s'"), how='left').set_index('zip')
            mask = Z['drivetime'].isnull()  # zips without a ZTCA
            Z = Z[['latitude','longitude']]
            X = np.radians(Z[~mask])
            Y = np.radians(Z[mask])
            M = (
                pd.DataFrame(haversine_distances(X, Y), index=X.index, columns=Y.index) # haversine distance between pairs with and without ZCTA
                .idxmin()  # find nearest ZCTA
                .reset_index()
                .set_axis(['new_zip','zip'], axis=1)
                .prep()
                .merge(D)  # merge the drivetimes for that ZCTA
                .drop(columns='zip')
                .rename(columns={'new_zip':'zip'})
            )
            df = pd.concat([D,M], ignore_index=True)
            return df
        df, new = self.get(fcn, root/'geo/drivetimes', self.get_zips)
        return df

########################################################
################# get term information #################
########################################################
    def get_terms(self, show=False):
        def fcn():
            qry = f"""
select
    stvterm_code as term_code
    ,replace(stvterm_desc, ' ', '') as term_desc
    ,stvterm_start_date as start_date
    ,stvterm_end_date as end_date
    ,stvterm_fa_proc_yr as fa_proc_yr
    ,stvterm_housing_start_date as housing_start_date
    ,stvterm_housing_end_date as housing_end_date
    ,sobptrm_census_date as census_date
from
    {catalog}saturnstvterm as A
inner join
    {catalog}saturnsobptrm as B
on
    stvterm_code = sobptrm_term_code
where
    sobptrm_ptrm_code='1'
"""
            df = run(qry, show).set_index('term_code')
            df['stable_date'] = df['census_date'].apply(lambda x: x+pd.Timedelta(days=7+4-x.weekday())) # Friday of week following census
            return df
        df, new = self.get(fcn, data/'terms')
        return df


    def get_cycle(self, term_code, cycle_day=None, cycle_date=None, show=False):
        term_code = int(term_code)
        year = term_code // 100
        term_desc, stable_date = self.get_terms().loc[term_code,['term_desc','stable_date']]
        if cycle_day is None:
            if cycle_date is None:
                cycle_date = pd.Timestamp.now()
            cycle_date = min(pd.to_datetime(cycle_date), pd.Timestamp.now()).normalize()
            cycle_day = (stable_date - cycle_date).days
        cycle_date = (stable_date - pd.Timedelta(days=cycle_day)).date()
        stem = f'{term_code}_{cycle_date}_{"-" if cycle_day < 0 else "+"}{rjust(abs(cycle_day),3,0)}'
        return cycle_day, cycle_date, stable_date, stem, term_desc, year
#######################################################
############ process flags reports archive ############
#######################################################
    def get_spriden(self, show=False):
        # Get id-pidm crosswalk so we can replace id by pidm in flags below
        # GA's should not have permissions to run this because it can see pii
        if 'spriden' not in self:
            qry = f"""
            select distinct
                spriden_id as id,
                spriden_pidm as pidm
            from
                {catalog}saturnspriden as A
            where
                spriden_change_ind is null
                and spriden_activity_date between '2000-09-01' and '2025-09-01'
                and spriden_id REGEXP '^[0-9]+'
            """
            self.spriden = run(qry, show)
        return self.spriden


    def process_flags(self, show=False):
        # GA's should not have permissions to run this because it can see pii
        counter = 0
        divide = False
        for src in sorted(flags_raw.iterdir(), reverse=True):
            counter += 1
            if counter > 5:
                break
            a,b = src.name.lower().split('.')
            if b != 'xlsx' or 'melt' in a or 'admitted' not in a:
                print(a, 'SKIP')
                continue
            # Handles 2 naming conventions that were used at different times
            try:
                current_date = pd.to_datetime(a[:10].replace('_','-'))
                multi = True
            except:
                try:
                    current_date = pd.to_datetime(a[-6:])
                    multi = False
                except:
                    print(a, 'FAIL')
                    continue
            book = pd.ExcelFile(src, engine='openpyxl')
            # Again, handles the 2 different versions with different sheet names
            if multi:
                sheets = {sheet:sheet for sheet in book.sheet_names if sheet.isnumeric() and int(sheet) % 100 in [1,6,8]}
            else:
                sheets = {a[:6]: book.sheet_names[0]}
            for term_code, sheet in sheets.items():
                current_day, current_date, stable_date, stem, term_desc, year = self.get_cycle(term_code, cycle_date=current_date)
                def fcn():
                    df = (
                        self.get_spriden()
                        .assign(current_day=current_day, current_date=current_date)
                        .merge(book.parse(sheet).prep(), on='id', how='right')
                        .drop(columns=['id','last_name','first_name','mi','pref_fname','street1','street2','primary_phone','call_em_all','email'], errors='ignore')
                    )
                    return df
                if self.get(fcn, flags_prc/f'{term_code}/flags_{stem}', read=False, divide=False)[1]:
                    divide = True
                    counter = 0
                    dst = flags_prc/f'flags_{year}.parquet'
                    dst.unlink(missing_ok=True)
        if divide:
            print(divider)
            self.combine_flags()


    def combine_flags(self, show=False):
        def fcn(year):
            L = [pd.read_parquet(src) for path in flags_prc.iterdir() if path.is_dir() and str(year) in path.stem for src in path.glob('*.parquet')]
            df = pd.concat(L, ignore_index=True).prep()
            del L
            for k in ['dob',*df.filter(like='date').columns]:  # convert date columns
                if k in df:
                    df[k] = pd.to_datetime(df[k], errors='coerce')
            return df
        for year in {int(x.stem)//100 for x in flags_prc.iterdir() if x.is_dir()}:
            self.get(fcn, flags_prc/f'flags_{year}', read=False, year=year)


    def get_flags(self, show=False):
        def fcn():
            df = (
                pd.read_parquet(flags_prc/f'flags_{self.year}.parquet')
                .query(f"current_date<='{self.cycle_date}'")
                .sort_values(['pidm','current_date'])
                .drop_duplicates(subset=['pidm','term_code'], keep='last')
            )
            df.loc[~df['state'].isin(self.get_states()),'zip'] = pd.NA
            df['zip'] = df['zip'].str.split('-', expand=True)[0].str[:5].to_numeric(errors='coerce')
            return df
        df, new = self.get(fcn, 'flags', self.combine_flags)
        return df
##########################################
############ get student data ############
##########################################
    def newest(self, qry, part, tbl='', sel=''):
        """The OPEIR daily snapshot experienced occasional glitched causing incomplete copies.
        Consequently, record can vanished then reappear later. This function fixes this issue."""
        part = join(part, ', ')
        if tbl == '':
            tbl = qry
        if sel != '':
            sel = ','+join(sel)

        qry = f"""
select
    {part}
    ,current_date
    ,min(current_date) over (partition by {part}) as first_date  --first date this record appeared
    ,max(current_date) over (partition by {part}) as last_date  --last date this record appeared
    ,least(greatest(timestamp('{self.cycle_date}'), min(current_date) over ()), max(current_date) over ()) as cycle_date  --clip cycle date between first & last date of ANY record
from
    {qry.strip()}
qualify
    cycle_date between first_date and last_date  -- keep records where cycle_date falls between that record's first & last appearance (+5 days for safety)
"""

        qry = f"""
select
    *
from {subqry(qry)}
where
    current_date <= '{self.cycle_date}'  -- discard records after cycle_date
qualify
    row_number() over (partition by {part} order by current_date desc) = 1  -- keep most recent remaining record
"""

        qry = f"""
select distinct
    pidm
    ,cycle_date
    ,current_date
    ,first_date
    ,last_date
    ,{get_desc('term_code')}
    ,{get_desc('levl_code')}
    ,{get_desc('styp_code')}
    ,{get_desc('camp_code')}
    ,{get_desc('coll_code_1')}
    ,{get_desc('dept_code')}
    ,{get_desc('majr_code_1')}
    --,gender
    ,spbpers_sex as gender
    ,birth_date
    ,{get_desc('spbpers_lgcy_code')}
    ,gorvisa_vtyp_code is not null as international
    ,gorvisa_natn_code_issue as natn_code, (select stvnatn_nation from {catalog}saturnstvnatn where gorvisa_natn_code_issue = stvnatn_nation limit 1) as natn_desc
    ,{coalesce('race_asian')}
    ,{coalesce('race_black')}
    ,coalesce(spbpers_ethn_cde=2, False) as race_hispanic
    ,{coalesce('race_native')}
    ,{coalesce('race_pacific')}
    ,{coalesce('race_white')}
    {indent(sel)}
from {subqry(qry)} as A

left join
    {tbl}
using
    ({part}, current_date)

left join
    {catalog}spbpers_v
on
    pidm = spbpers_pidm

left join (
    select
        *
    from
        {catalog}generalgorvisa
    qualify
        row_number() over (partition by gorvisa_pidm order by gorvisa_seq_no desc) = 1
    )
on
    pidm = gorvisa_pidm

left join (
    select
        gorprac_pidm
        ,max(gorprac_race_cde='AS') as race_asian
        ,max(gorprac_race_cde='BL') as race_black
        ,max(gorprac_race_cde='IN') as race_native
        ,max(gorprac_race_cde='HA') as race_pacific
        ,max(gorprac_race_cde='WH') as race_white
    from
        {catalog}generalgorprac
    group by
        gorprac_pidm
    )
on
    pidm = gorprac_pidm
"""
        return qry


    def get_admissions(self, show=False):
        def fcn():
            # tbl = f'dev.opeir.opeiradmissions_{self.term_desc}'
            tbl = f'dev.opeir.admissions_{self.term_desc}_v'
            qry = self.newest(
                tbl = tbl,
                part = ['pidm', 'appl_no'],
                sel = [
                    'appl_no',
                    get_desc('apst_code'),
                    get_desc('apdc_code'),
                    get_desc('admt_code'),
                    get_desc('saradap_resd_code'),
                    'hs_percentile',
                    # 'sbgi_code',
                ],
                qry = f"""
    {tbl} as A
inner join
    {catalog}saturnstvapdc as B
on
    apdc_code = stvapdc_code
where
    stvapdc_inst_acc_ind is not null  --only accepted""")
            df = run(qry, show)
            return df
        df, new = self.get(fcn, 'admissions')
        return df


    def get_registrations(self, show=False):
        def fcn():
            tbl = f'dev.opeir.opeirregistration_{self.term_desc}'
            if spark.catalog.tableExists(tbl):
                qry = self.newest(
                    tbl = tbl,
                    part = ['pidm','crn'],
                    sel = [
                        'credit_hr as count',
                        'subj_code || crse_numb as crse_code',
                        # 'crn',
                        # 'subj_code',
                        # 'crse_numb',
                    ],
                    qry = f"""
    {tbl} as A
where
    credit_hr > 0
    and subj_code <> 'INST'""")
                A = run(qry, show)
                B = A.groupb(['pidm','crse_code'])['count'].sum().reset_index('crse_code')
                C = B.groupb('pidm')[['count']].sum().assign(crse_code='_tot_sch')
                D = C.copy()
                B['count'] = 1
                D['count'] = 1
                D['crse_code'] = '_headcnt'
                E = pd.concat([B,C,D])
                F = A.drop(columns=['count','crse_code']).sort_values('current_date').groupb('pidm').last()
                df = F.join(E)
            else:
                # placeholder if table DNE
                df = pd.DataFrame(columns=['pidm','levl_code','styp_code','count','crse_code']).set_index('pidm')
            df.get_duplicates(['pidm','crse_code'])
            return df
        df, new = self.get(fcn, 'registrations')
        return df

# self = Term(
#     cycle_day=0,
#     term_code=202408,
#     overwrite=[
#         # 'terms',
#         # 'zips',
#         # 'drivetimes',
#         # 'flags',
#         # 'admissions',
#         'registrations',
#     ]
# )
# self.process_flags()
# self.combine_flags()
# # self.get_zips()
# # self.get_drivetimes()
# self.get_registrations(show=True)
# self.get_admissions(show=True)
# # A = self.get_admissions(show=True)
# # self.get_students()
# self.get_terms().loc[202501]


def prediction(clf, X, y):
    Z = X.copy()
    Z['prediction'] = clf.predict_proba(X)[:,1]
    Z['actual'] = y
    Z['error'] = Z['prediction'] - Z['actual']
    Z['mse'] = Z['error']**2
    Z['mae'] = np.abs(Z['error'])
    Z['log_loss'] = -1*(Z['actual']*np.log(Z['prediction']) + (1-Z['actual'])*np.log(1-Z['prediction']))
    return Z
setmeth(fl.automl.automl.AutoML, prediction)


@dataclasses.dataclass
class AMP(Term):
    cycle_date: str
    crse_code: str = '_headcnt'
    idx: str = 'styp_desc'
    agg: tuple = ('styp_desc','camp_desc')
    flaml: dict = None
    overwrite: set = None

    def __post_init__(self):
        self.cycle_date = min(pd.to_datetime(self.cycle_date), pd.Timestamp.now()).normalize()
        self.cycle_date -= pd.Timedelta(days=self.cycle_date.weekday()-2)  # move to closest Wednesday (Flags release)
        self.year = self.cycle_date.year
        self.term_code = 100*self.year+8
        super().__post_init__()
        kwargs = {k: self[k] for k in ['cycle_date','term_code','overwrite']}
        self.current = Term(**kwargs)
        self.summer  = Term(**kwargs | {'term_code': 100*self.year+6})
        self.stable  = Term(**kwargs | {'cycle_day': 0})
        self.overwrite |= {f"{k}_{self.crse_code}" for k in self.overwrite.intersection(['prepared','models','predictions','forecasts'])}
        self.flaml = dict() if self.flaml is None else self.flaml
        self.features = {
            'act_equiv':pd.NA,
            'age':pd.NA,
            'camp_desc':'stephenville',
            'drivetime':pd.NA,
            'eager':pd.NA,
            'fafsa': False,
            'finaid': False,
            'gap_score':0,
            'gender':pd.NA,
            'hs_qrtl':pd.NA,
            'lgcy':False,
            'oriented':False,
            'race_asian':False,
            'race_black':False,
            'race_hispanic':False,
            'race_native':False,
            'race_pacific':False,
            'race_white':False,
            'schlship':False,
            'ssb':False,
            'tsi_math':False,
            'tsi_reading':False,
            'tsi_writing':False,
            'verified':False,
            'waiver':False,
        }
        # self.get_raw()
        # self.get_imputed()
        # self.get_prepared()
        # if self.year < max(years):
        #     self.get_models()


    def get_raw(self):
        def fcn():
            df = pd.concat([x.get_admissions() for x in [self.current, self.summer]], ignore_index=True)
            mask = df.groupby('pidm', group_keys=False).apply(lambda x: (x['levl_code']=='ug').all() & (x['appl_no']==x['appl_no'].max()))
            df = (
                df.loc[mask]
                .query("levl_code=='ug' & styp_code in ['n','r','t']")
                .merge(self.current.get_flags(), on=['pidm','term_code'], how='left', suffixes=['', '_flags'])
                .merge(self.current.get_drivetimes(), on=['zip','camp_code'], how='left', suffixes=['', '_zips'])
                .prep()
                .set_index('pidm')
                .sort_index()
            )
            for c in ['gap_score','t_gap_score','ftic_gap_score']:
                if c not in df:
                    df[c] = pd.NA
            df['gap_score'] = np.where(
                df['styp_code']=='n',
                df['ftic_gap_score'].combine_first(df['t_gap_score']).combine_first(df['gap_score']),
                df['t_gap_score'].combine_first(df['ftic_gap_score']).combine_first(df['gap_score']))
            df['oriented'] = df['orientation_hold_exists'].isnull() | df['orien_sess'].notnull() | df['registered'].notnull()
            df['verified'] = df['selected_for_ver'].isnull() | df['ver_complete'].notnull()
            df['sat10_total_score'] = (36-9) / (1600-590) * (df['sat10_total_score']-590) + 9
            df['act_equiv'] = df[['act_new_comp_score','sat10_total_score']].max(axis=1)
            df['eager'] = (self.stable_date - df['first_date']).dt.days
            df['age'] = (self.stable_date - df['birth_date']).dt.days
            for k in ['reading', 'writing', 'math']:
                df[f'tsi_{k}'] = ~df[k].isin(['not college ready', 'retest required', pd.NA, None, np.nan])
            repl = {'ae':0, 'n1':1, 'n2':2, 'n3':3, 'n4':4, 'r1':1, 'r2':2, 'r3':3, 'r4':4}
            df['hs_qrtl'] = pd.cut(df['hs_pctl'], bins=[-1,25,50,75,90,101], labels=[4,3,2,1,0], right=False).combine_first(df['apdc_code'].map(repl))
            df['lgcy'] = ~df['lgcy_code'].isin(['o',pd.NA,None,np.nan])
            df['resd'] = df['resd_code'] == 'r'
            for k in ['waiver_desc','fafsa_app','ssb_last_accessed','finaid_accepted','schlship_app']:
                df[k.split('_')[0]] = df[k].notnull()

            # df[list(self.features)].get_missing()
            df.get_duplicates()
            df.query('current_date_flags.isnull()')['first_date'].value_counts().sort_index().disp(10)
            return df
        
        df, new = self.get(fcn, f'raw', [
            self.current.get_admissions,
            self.summer .get_admissions,
            self.current.get_flags,
            self.current.get_drivetimes])
        return df


    def get_imputed(self):
        def fcn():
            def fcn1(df):
                feat = listify(self.features.keys())
                idx = difference(listify(self.idx)+listify(self.agg), feat)
                X = df.fillna(self.features).prep(category=True).set_index(idx, append=True)[feat]
                imp = mf.ImputationKernel(X.reset_index(drop=True), random_state=self.seed)
                imp.mice(10)
                XX = imp.complete_data().set_index(X.index)
                XX.get_missing()
                return XX
            return {key: fcn1(df) for key, df in self.get_raw().groupby(self.idx)}
        dct, new = self.get(fcn, f'imputed', self.get_raw, suffix='.pkl')
        return dct


    def get_prepared(self):
        def fcn():
            g = lambda k, v, n=None: self[k].get_registrations().query(f"crse_code=='{v}'")['count'].rename(k if n is None else n)
            Z = {key: X
                .join(g('current', '_tot_sch', '_tot_sch'))
                .join(g('current', self.crse_code))
                .join(g('stable' , self.crse_code))
                .fillna({'_tot_sch':0, 'current':0, 'stable':0})
                for key, X in self.get_imputed().items()}
            {key: z.get_duplicates() for key, z in Z.items()}
            return {key: [X, X.pop('stable')] for key, X in Z.items()}
        dct, new = self.get(fcn, f'prepared_{self.crse_code}', [self.get_imputed, self.current.get_registrations, self.stable.get_registrations], suffix='.pickle')
        return dct


    def get_enrollments(self, key='crse_code'):
        grp = unique('crse_code',*listify(self.idx),key)
        g = lambda X: X.query(f"levl_code=='ug' & styp_code in ['n','t','r']").groupb(grp)['count'].sum()
        S = self['stable'].get_registrations()
        df = pd.DataFrame()
        df['current'] = g(self.get_raw()[[]].join(S, how='inner'))
        df['actual'] = g(S)
        df = df.fillna(0)
        df['mlt'] = df['actual'] / df['current']
        return df.sort_index()


    def get_models(self):
        def fcn():
            def fcn1(Z):
                dct = {
                    'time_budget':60,
                    'task':'classification',
                    'verbose':0,
                    'metric':'log_loss',
                    'eval_method':'cv',
                    'n_splits':3,
                    'seed':self.seed,
                    # 'early_stop':True,
                    'estimator_list': ['xgboost','lgbm','rf'],
                } | self.flaml
                clf = fl.AutoML(**dct)
                clf.fit(*Z, **dct)
                return clf
            return {key: fcn1(Z) for key, Z in self.get_prepared().items()}
        clf, new = self.get(fcn, f'models_{self.crse_code}', self.get_prepared, suffix='.pickle')
        return clf


    def get_predictions(self, learners=dict()):
        def fcn():
            L = [
                clf.prediction(*self.get_prepared()[key])
                .assign(
                    crse_code=self.crse_code,
                    prediction_year=self.year,
                    learner_year=learner.year,
                    mlt=learner.get_enrollments().loc[self.crse_code,*listify(key)]['mlt'],
                )
                for year, learner in learners.items() if 'models' in learner
                for key, clf in learner.get_models().items()]
            return pd.concat(L).prep(category=True) if len(L)>0 else pd.DataFrame()
        df, new = self.get(fcn, f'predictions_{self.crse_code}', self.get_prepared)
        return df
    

    def get_forecasts(self):
        def fcn():
            err = ['mse','mae','log_loss']
            agg = {'prediction':'sum'}
            if 'models' in self:
                agg |= {k:'mean' for k in err}

            def fcn1(key):
                df = (
                    self.get_predictions()
                    .assign(prediction=lambda Z: Z['prediction']*Z['mlt'])
                    .groupb(unique('crse_code','prediction_year','learner_year',*listify(self.idx),key))
                    .agg(agg)
                )
                df['prediction'] = df['prediction'].round()
                if 'models' in self:
                    df = df.join(self.get_enrollments(key)['actual']).fillna(0)
                    df['error'] = df['prediction'] - df['actual']
                    df['error_pct'] = (df['error'] / df['actual'] * 100).round(1)
                    df = df[['prediction','actual','error','error_pct',*err]]
                return df.prep().sort_index()
            return {key: fcn1(key) for key in self.agg}
        dct, new = self.get(fcn, f'forecasts_{self.crse_code}', self.get_predictions, suffix='.pickle')
        return dct

kwargs = {
    'overwrite': {
        # 'terms',
        # 'zips',
        # 'drivetimes',
        # 'flags',
        # 'admissions',
        # 'registrations',
        # 'raw',
        # 'imputed',
        # 'models',
        'predictions',
        'forecasts',
        },
    'flaml': {
        'time_budget': 10,
    },
}

# cycle_date = '2025-01-02'
cycle_date = pd.Timestamp.now()
years = [2024,2025]
crse_codes = [
    '_headcnt',
    'math1314',
]

years = listify(years)
cycle_date = min(pd.to_datetime(cycle_date), pd.Timestamp.now()).normalize()
cycle_date -= pd.Timedelta(days=(cycle_date.weekday()-2)%7)  # move to preceeding Wednesday (flags release)

amps = dict()
for crse_code in crse_codes:
    print(crse_code)
    amps[crse_code] = {year: AMP(crse_code=crse_code, cycle_date=cycle_date.replace(year=year), **kwargs) for year in years}
    for year, self in amps[crse_code].items():
        print(year)
        # self.get_prepared()
        if self.year < max(years):
            self.get_models()
        {kwargs['overwrite'].discard(k) for k in {'terms','zips','drivetimes'}}
    {kwargs['overwrite'].discard(k) for k in {'flags','admissions','registrations','raw','imputed'}}
    for year, self in amps[crse_code].items():
        self.get_predictions(amps[crse_code]).disp(2)
        for k,v in self.get_forecasts().items():
            v.disp()
    # # amps[crse_code] = dct

In [0]:
## useful old code
#     def get_registrations(self, overwrite=False, show=False):
#         def fcn():
#             dct = {
#                 'sfrstcr_pidm':'pidm',
#                 'ssbsect_term_code':'term_code',
#                 'sgbstdn_levl_code':'levl_code',
#                 'sgbstdn_styp_code':'styp_code',
#                 'ssbsect_crn':'crn',
#             }
#             qry = f"""
# select
#     {indent(join(alias(dct)))}
#     ,lower(ssbsect_subj_code) || ssbsect_crse_numb as crse_code
#     ,max(ssbsect_credit_hrs) as credit_hr
# from
#     {catalog}saturnsfrstcr as A
# inner join
#     {catalog}saturnssbsect as B
# on
#     sfrstcr_term_code = ssbsect_term_code
#     and sfrstcr_crn = ssbsect_crn
# inner join (
#     select
#         *
#     from
#         {catalog}sgbstdn_amp_v
#     where
#         sgbstdn_term_code_eff <= {self.term_code}
#     qualify
#         row_number() over (partition by sgbstdn_pidm order by sgbstdn_term_code_eff desc) = 1
#     ) as C
# on
#     sfrstcr_pidm = sgbstdn_pidm
# where
#     sfrstcr_term_code = {self.term_code}
#     and sfrstcr_ptrm_code not in ('28','R3') -- drop weird term part
#     and sfrstcr_add_date <= '{self.cycle_date}' -- added before cycle_day
#     and (sfrstcr_rsts_date > '{self.cycle_date}' or sfrstcr_rsts_code in ('DC','DL','RD','RE','RW','WD','WF')) -- dropped after cycle_day or still enrolled
#     and ssbsect_subj_code <> 'INST' -- exceptional sections
# group by
#     {indent(join(dct.keys()))}
#     ,ssbsect_subj_code
#     ,ssbsect_crse_numb
# """

#             qry = f"""
# with A as {subqry(qry)}
# select * from A

# union all

# select
#     {indent(join(dct.values()))}
#     ,'_allcrse' as crse_code
#     ,sum(credit_hr) as credit_hr
# from A
# group by
#     {indent(join(dct.values()))}

# union all

# select
#     {indent(join(dct.values()))}
#     ,'_anycrse' as crse_code
#     ,case when sum(credit_hr) > 0 then 1 else 0 end as credit_hr
# from A
# group by
#     {indent(join(dct.values()))}
# """
#             df = run(qry, show)
#             return df
#         df, new = self.get(fcn, 'registrations', overwrite)
#         return df


#     def get_registrations(self, overwrite=False, show=False):
#         def fcn():
#             dct = {
#                 'sfrstcr_pidm':'pidm',
#                 'ssbsect_term_code':'term_code',
#             }
#             qry = f"""
# select
#     {indent(join(alias(dct)))}
#     ,lower(ssbsect_subj_code) || ssbsect_crse_numb as crse_code
#     ,max(ssbsect_credit_hrs) as credit_hr
# from
#     {catalog}saturnsfrstcr as A
# inner join
#     {catalog}saturnssbsect as B
# on
#     sfrstcr_term_code = ssbsect_term_code
#     and sfrstcr_crn = ssbsect_crn
# where
#     sfrstcr_term_code = {self.term_code}
#     and sfrstcr_error_flag is null
#     and sfrstcr_ptrm_code not in ('28','R3') -- drop weird term part
#     and sfrstcr_add_date <= '{self.cycle_date}' -- added before cycle_day
#     and (sfrstcr_rsts_date > '{self.cycle_date}' or sfrstcr_rsts_code in ('DC','DL','RD','RE','RW','WD','WF')) -- dropped after cycle_day or still enrolled
#     and ssbsect_subj_code <> 'INST' -- exceptional sections
# group by
#     {indent(join(dct.keys()))}
#     ,ssbsect_subj_code
#     ,ssbsect_crse_numb
# """

#             qry = f"""
# with A as {subqry(qry)}
# select * from A

# union all

# select
#     {indent(join(dct.values()))}
#     ,'_allcrse' as crse_code
#     ,sum(credit_hr) as credit_hr
# from A
# group by
#     {indent(join(dct.values()))}

# union all

# select
#     {indent(join(dct.values()))}
#     ,'_anycrse' as crse_code
#     ,case when sum(credit_hr) > 0 then 1 else 0 end as credit_hr
# from A
# group by
#     {indent(join(dct.values()))}
# """
#             df = run(qry, show)
#             return df
#         df, new = self.get(fcn, 'registrations', overwrite)

#         S = self.students[['pidm','term_code','styp_code']]
#         return df



    #     def fcn():
#             qry = f"""
# select
#     sfrstcr_pidm as pidm
#     ,sfrstcr_term_code as term_code
#     ,lower(ssbsect_subj_code) || ssbsect_crse_numb as crse_code
#     ,max(ssbsect_credit_hrs) as credit_hr
# from
#     {catalog}saturnsfrstcr as A
# inner join
#     {catalog}saturnssbsect as B
# on
#     sfrstcr_term_code = ssbsect_term_code
#     and sfrstcr_crn = ssbsect_crn
# where
#     sfrstcr_term_code = {self.term_code}
#     --and sfrstcr_error_flag is null
#     and sfrstcr_ptrm_code not in ('28','R3') -- drop weird term part
#     and sfrstcr_add_date <= '{self.cycle_date}' -- added before cycle_day
#     and (sfrstcr_rsts_date > '{self.cycle_date}' or sfrstcr_rsts_code in ('DC','DL','RD','RE','RW','WD','WF')) -- dropped after cycle_day or still enrolled
#     and ssbsect_subj_code <> 'INST' -- exceptional sections
# group by
#     sfrstcr_pidm
#     ,sfrstcr_term_code
#     ,ssbsect_subj_code
#     ,ssbsect_crse_numb
# """

#             qry = f"""
# with A as {subqry(qry)}
# select * from A

# union all

# select
#     pidm
#     ,term_code
#     ,'_allcrse' as crse_code
#     ,sum(credit_hr) as credit_hr
# from A
# group by
#     pidm
#     ,term_code

# union all

# select
#     pidm
#     ,term_code
#     ,'_anycrse' as crse_code
#     ,case when sum(credit_hr) > 0 then 1 else 0 end as credit_hr
# from A
# group by
#     pidm
#     ,term_code
# """


# don't delete - could be useful & was hard to create
            # stat_codes = ['AL','AR','AZ','CA','CO','CT','DC','DE','FL','GA','IA','ID','IL','IN','KS','KY','LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VA','VT','WA','WI','WV','WY'] # not AK & HI b/c can't get driving distance
#     ,{get_desc('spraddr_cnty_code')[0]}
#     ,{get_desc('spraddr_stat_code')[0]}
#     ,zip_code

# left join (
#     select
#         *
#         ,try_to_number(left(spraddr_zip, 5), '00000') as zip_code
#         ,case
#             when spraddr_atyp_code = 'PA' then 6
#             when spraddr_atyp_code = 'PR' then 5
#             when spraddr_atyp_code = 'MA' then 4
#             when spraddr_atyp_code = 'BU' then 3
#             when spraddr_atyp_code = 'BI' then 2
#             when spraddr_atyp_code = 'P1' then 1
#             when spraddr_atyp_code = 'P2' then 0
#             end as spraddr_atyp_rank
#     from
#         {catalog}spraddr_amp_v
#     where
#         spraddr_stat_code in ('{join(stat_codes, "','")}')
#         and spraddr_zip is not null
#     qualify
#         row_number() over (partition by spraddr_pidm order by spraddr_atyp_rank desc, spraddr_seqno desc) = 1
# )
# on
#     pidm = spraddr_pidm

# {get_desc('spraddr_cnty_code')[1]}
# {get_desc('spraddr_stat_code')[1]}



    # def get_zips(self, show=False):
    #     """takes ~3 hours toget zip codes and find nearest point on road network to the provided representative point"""
    #     def fcn():
    #         from pgeocode import Nominatim
    #         nomi = Nominatim('us')
    #         df = nomi.query_postal_code(pd.Series(nomi._data['postal_code'])).query("state_code.notnull() & state_code not in ['AK', 'HI', 'MH']").prep().set_index('postal_code').rename_axis('zip')
    #         nearest = lambda x: join(requests.get(f"http://router.project-osrm.org/nearest/v1/driving/{x['longitude']},{x['latitude']}").json()['waypoints'][0]['location'],',')
    #         df['point'] = df.apply(nearest, axis=1)
    #         return df
    #     df, new = self.get(fcn, root/'zips')
    #     self.states = set(df['state_code'])
    #     return df


    # def get_drivetimes(self, show=False):
    #     def fcn():
    #         campus_coords = {
    #             's': [-98.215784,32.216217],
    #             # 'm': '-97.432975,32.582436',
    #             # 'w': 76708,
    #             # 'r': 77807,
    #             }

    #         url = "https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_zcta520_500k.zip"
    #         gdf = gpd.read_file(url).prep().set_index('zcta5ce20').iloc[:5]
    #         pts = gdf.sample_points(size=5,method="uniform").explode()#.apply(lambda geom: f"{geom.x},{geom.y}")
    #         df = pts.to_frame()[[]]
    #         url = "http://router.project-osrm.org/table/v1/driving"
    #         headers = {"Content-Type": "application/json"}
    #         for k, v in campus_coords.items():
    #             u = [v, *pts]
    #             print(u)
    #             data = {
    #                 "coordinates": u,
    #                 "annotations": ["duration", "distance"],
    #                 "sources": 0,
    #             }
    #             response = requests.post(url, json=data, headers=headers)
    #             print(response.json())
    #             assert 1==2

            # for k, v in campus_coords.items():
            #     u = join([v, *pts], ';')
            #     url = f"http://router.project-osrm.org/table/v1/driving/{u}?destinations={0}&annotations=duration,distance"
            #     print(url)
            #     print(requests.get(url))
            #     df[k] = np.squeeze(requests.get(url).json()['durations'])[1:]/60
    #         # df.disp(10)
    #         # df = df.groupby('zip').min()
    #         # df.disp(10)
    #         df = df.groupby(level=0).min().stack().reset_index().set_axis(['zip','camp_code','drivetime'], axis=1)
    #         return df

            # df = self.zips.iloc[34339:34349].copy()
            # u = join(df.apply(lambda x: f"{x['longitude']},{x['latitude']}", axis=1),';')
            # for k, z in campus_zips.items():
            #     df[k] = np.squeeze(
            #     requests.get(f"http://router.project-osrm.org/table/v1/driving/{u}?destinations={df.index.get_loc(z)}&annotations=duration,distance&fallback_speed=26.8&fallback_coordinate=snapped"
            #     ).json()['distances'])/1609

            # for k, z in campus_zips.items():
            #     df[k] = np.squeeze(
            #     requests.get(f"http://router.project-osrm.org/table/v1/driving/{u}?destinations={df.index.get_loc(z)}&annotations=duration,distance&fallback_speed=26.8&fallback_coordinate=snapped"
            #     ).json()['durations'])/60

            # self.zips = self.zips.iloc[34339:34349]
            # self.zips.disp(20)
            # u = join(self.zips['point'],';')
            
            # dct = dict()
            # for k, z in campus_zips.items():
            #     i = self.zips.index.get_loc(z)
            #     print(self.zips.iloc[i])
            #     url = f"http://router.project-osrm.org/table/v1/driving/{u}?destinations={0}&annotations=duration,distance&fallback_speed=26.8&fallback_coordinate=snapped"
            #     print(url)
            #     response = requests.get(url).json()
            #     for k,v in response.items():
            #         print(k)
            #         display(v)
            #         print()
            #     print(response['distance'])
            #     dct[k] = np.squeeze(response['durations'])

            # # dct = {k: np.squeeze(
            # #     requests.get(f"http://router.project-osrm.org/table/v1/driving/{u}?destinations={self.zips.index.get_loc(z)}&fallback_speed=600&fallback_coordinate=snapped"
            # #     ).json()['durations'])/60 for k, z in campus_zips.items()}
            # dct = {k: np.squeeze(
            #     requests.get(f"http://router.project-osrm.org/table/v1/driving/{u}?destinations={self.zips.index.get_loc(z)}&annotations=duration,distance&fallback_speed=26.8&fallback_coordinate=snapped"
            #     ).json()['distances']) for k, z in campus_zips.items()}

            # df = pd.DataFrame(dct, index=self.zips.index).stack().rename_axis(['zip','camp_code']).rename('drivetime') / 1609
            # return df
            # print()
            # dct = {k: self.zips.loc[y] for k, y in {
            #     's': 76402,
            #     'm': 76036,
            #     'w': 76708,
            #     'r': 77807,
            #     }.items()}
            # L = [
            #     self.get(
            #         lambda: X.apply(get_driving_distance, y=y, axis=1).rename('distance').reset_index().assign(camp_code=k),
            #         root/f'distances/distances_{s}_{k}',
            #         divide=False,
            #     )[0] for s, X in self.zips.groupby('state_code') for k, y in dct.items()]
            # return pd.concat(L, ignore_index=True)
        # df, new = self.get(fcn, root/'drivetimes')#, self.get_zips)
        # return df




    # def get_flags_history(self, cutoff=202206):
    #     def fcn():
    #         import pyarrow.parquet as pq
    #         print()
    #         L = []
    #         for path in sorted(flags_prc.iterdir(), reverse=True):
    #             print(path)
    #             for src in path.iterdir():
    #                 _, term_code, cycle_date, cycle_day = src.stem.split('_')
    #                 col = pq.ParquetFile(src).schema.names
    #                 df = pd.DataFrame(columns=col).assign(term_code=[int(term_code)], cycle_date=[cycle_date]).fillna(True)
    #                 L.append(df)
    #         df = pd.concat(L).fillna(False).set_index(['cycle_date','term_code']).sort_index()
    #         return df[sorted(df.columns)]
    #     df, new = self.get(fcn, path=data/'flags_history')
    #     A = df.query(f'term_code>={cutoff}').groupby('term_code').sum().sort_index(ascending=False).T.rename_axis('variable')
    #     B = A == A.max()
    #     B.insert(0, 'n', B.sum(axis=1))
    #     return B.reset_index().sort_values(['n', 'variable'], ascending=[False, True])



############ annoying warnings to suppress ############
# [warnings.filterwarnings(action='ignore', message=f".*{w}.*") for w in [
#     "Could not infer format, so each element will be parsed individually, falling back to `dateutil`",
#     "Engine has switched to 'python' because numexpr does not support extension array dtypes",
#     "The default of observed=False is deprecated and will be changed to True in a future version of pandas",
#     "errors='ignore' is deprecated"
#     "The behavior of DataFrame concatenation with empty or all-NA entries is deprecated",
#     "The behavior of array concatenation with empty entries is deprecated",
#     "DataFrame is highly fragmented",
# ]]



# for fore in amp.values():
#     for base in amp.values():
#         if base.year < max(years):
#             for styp, clf in base.get_models().items():
#                 for k in ['predictions','headcounts']:
#                     fore.__dict__.setdefault(k, dict()).setdefault(styp, dict())
#                 y = clf.prediction(fore.get_prepared()[styp])
#                 fore.predictions[styp][base.year] = y
#                 s = (
#                     (y[['pred']].sum() * base.get_enrollments().loc[base.crse_code,styp]['mlt']).round()
#                     .rename(base.year).to_frame().T.rename_axis('base_year')
#                     .assign(styp_code=styp, forecast_year=fore.year)
#                     .reset_index().set_index(['styp_code','forecast_year','base_year'])
#                 )
#                 if fore.year < max(years):
#                     s['true'] = fore.get_enrollments().loc[base.crse_code,styp]['stable']
#                     s['error'] = s['pred'] - s['true']
#                     s['error_pct'] = round(s['error'] / s['true'] * 100, 2)
#                 fore.headcounts[styp][base.year] = s.prep()
#     fore.forecasts = {styp: pd.concat(v.values()) for styp, v in fore.headcounts.items()}
# amp[2023].forecasts['n']



# def get_desc(code):
#     for nm in code.split('_'):
#         if len(nm) == 4:
#             break
#     return [f'{code} as {nm}_code, stv{nm}_desc as {nm}_desc', f'left join {catalog}saturnstv{nm} on {code} = stv{nm}_code']


#             qry = f"""
# select
#     pidm
#     ,{self.cycle_day} as cycle_day
#     ,timestamp('{self.cycle_date}') as cycle_date
#     ,current_date
#     ,first_date
#     ,final_date
#     ,{get_desc('term_code')[0]}
#     ,appl_no
#     ,{get_desc('apst_code')[0]}
#     ,{get_desc('apdc_code')[0]}
#     ,{get_desc('admt_code')[0]}
#     ,{get_desc('wrsn_code')[0]}
#     ,{get_desc('levl_code')[0]}
#     ,{get_desc('styp_code')[0]}
#     ,{get_desc('camp_code')[0]}
#     ,{get_desc('coll_code_1')[0]}
#     ,{get_desc('dept_code')[0]}
#     ,{get_desc('majr_code_1')[0]}
#     ,{get_desc('saradap_resd_code')[0]}
#     ,gender
#     ,birth_date
#     ,{get_desc('spbpers_lgcy_code')[0]}
#     ,gorvisa_vtyp_code is not null as international
#     ,gorvisa_natn_code_issue as natn_code, stvnatn_nation as natn_desc
#     ,{coalesce('race_asian')}
#     ,{coalesce('race_black')}
#     ,coalesce(spbpers_ethn_cde=2, False) as race_hispanic
#     ,{coalesce('race_native')}
#     ,{coalesce('race_pacific')}
#     ,{coalesce('race_white')}
#     ,hs_percentile
#     ,sbgi_code
#     ,enrolled_ind='Y' as enrolled_ind

# from {subqry(qry)} as A

# left join
#     {catalog}spbpers_amp_v
# on
#     pidm = spbpers_pidm

# left join (
#     select
#         *
#     from
#         {catalog}generalgorvisa
#         --{catalog}gorvisa_amp_v
#     qualify
#         row_number() over (partition by gorvisa_pidm order by gorvisa_seq_no desc) = 1
#     )
# on
#     pidm = gorvisa_pidm

# left join (
#     select
#         gorprac_pidm
#         ,max(gorprac_race_cde='AS') as race_asian
#         ,max(gorprac_race_cde='BL') as race_black
#         ,max(gorprac_race_cde='IN') as race_native
#         ,max(gorprac_race_cde='HA') as race_pacific
#         ,max(gorprac_race_cde='WH') as race_white
#     from
#         {catalog}generalgorprac
#     group by
#         gorprac_pidm
#     )
# on
#     pidm = gorprac_pidm

# {get_desc('term_code')[1]}
# {get_desc('levl_code')[1]}
# {get_desc('styp_code')[1]}
# {get_desc('admt_code')[1]}
# {get_desc('wrsn_code')[1]}
# {get_desc('apst_code')[1]}
# {get_desc('apdc_code')[1]}
# {get_desc('camp_code')[1]}
# {get_desc('coll_code_1')[1]}
# {get_desc('dept_code')[1]}
# {get_desc('majr_code_1')[1]}
# {get_desc('saradap_resd_code')[1]}
# {get_desc('gorvisa_natn_code_issue')[1]}
# {get_desc('spbpers_lgcy_code')[1]}

# qualify
#     min(levl_code='UG') over (partition by pidm) = True  -- remove pidm's with graduate admission even it if also has an undergradute admission, min acts like logical and
#     and row_number() over (partition by pidm order by appl_no desc) = 1  -- de-duplicate the few remaining pidms with multiple record by keeping highest appl_no
# """



#     def get_registrations(self, show=False):
#         def fcn():
#             dct = {
#                 'sfrstcr_pidm':'pidm',
#                 'ssbsect_term_code':'term_code',
#                 'sgbstdn_levl_code':'levl_code',
#                 'sgbstdn_styp_code':'styp_code',
#             }
#             qry = f"""
# select
#     {indent(join(alias(dct)))}
#     ,lower(ssbsect_subj_code) || ssbsect_crse_numb as crse_code
#     ,max(ssbsect_credit_hrs) as credit_hr
# from
#     {catalog}saturnsfrstcr as A
# inner join
#     {catalog}saturnssbsect as B
# on
#     sfrstcr_term_code = ssbsect_term_code
#     and sfrstcr_crn = ssbsect_crn
# inner join (
#     select
#         *
#     from
#         {catalog}sgbstdn_amp_v
#     where
#         sgbstdn_term_code_eff <= {self.term_code}
#     qualify
#         row_number() over (partition by sgbstdn_pidm order by sgbstdn_term_code_eff desc) = 1
#     ) as C
# on
#     sfrstcr_pidm = sgbstdn_pidm
# where
#     sfrstcr_term_code = {self.term_code}
#     and sfrstcr_error_flag is null
#     and sfrstcr_ptrm_code not in ('28','R3') -- drop weird term part
#     and sfrstcr_add_date <= '{self.cycle_date}' -- added before cycle_day
#     and (sfrstcr_rsts_date > '{self.cycle_date}' or sfrstcr_rsts_code in ('DC','DL','RD','RE','RW','WD','WF')) -- dropped after cycle_day or still enrolled
#     and ssbsect_subj_code <> 'INST' -- exceptional sections
#     and ssbsect_credit_hrs > 0
# group by
#     {indent(join(dct.keys()))}
#     ,ssbsect_subj_code
#     ,ssbsect_crse_numb
# """

#             qry = f"""
# with A as {subqry(qry)}
# select * from A

# union all

# select
#     {indent(join(dct.values()))}
#     ,'_allcrse' as crse_code
#     ,sum(credit_hr) as credit_hr
# from A
# group by
#     {indent(join(dct.values()))}

# union all

# select
#     {indent(join(dct.values()))}
#     ,'_anycrse' as crse_code
#     ,1 as credit_hr
# from A
# group by
#     {indent(join(dct.values()))}
# """
#             df = run(qry, show).set_index(['crse_code','pidm'])
#             return df
#         df, new = self.get(fcn, 'registrations')
#         return df


#     def get_registrations(self, show=False):
#         def fcn():
#             grp = join([
#                 'pidm',
#                 'term_code','term_desc',
#                 'levl_code','levl_desc',
#                 'styp_code','styp_desc',
#                 ], ', ')

#             qry = f"""
# select
#     sfrstcr_pidm as pidm
#     ,{get_desc('ssbsect_term_code')}
#     ,{get_desc('sgbstdn_levl_code')}
#     ,{get_desc('sgbstdn_styp_code')}
#     ,lower(ssbsect_subj_code) || ssbsect_crse_numb as crse_code
#     ,max(ssbsect_credit_hrs) as credit_hr
# from
#     {catalog}saturnsfrstcr as A
# inner join
#     {catalog}saturnssbsect as B
# on
#     sfrstcr_term_code = ssbsect_term_code
#     and sfrstcr_crn = ssbsect_crn
# inner join (
#     select
#         *
#     from
#         {catalog}sgbstdn_amp_v
#     where
#         sgbstdn_term_code_eff <= {self.term_code}
#     qualify
#         row_number() over (partition by sgbstdn_pidm order by sgbstdn_term_code_eff desc) = 1
#     ) as C
# on
#     sfrstcr_pidm = sgbstdn_pidm
# where
#     sfrstcr_term_code = {self.term_code}
#     and sfrstcr_error_flag is null
#     and sfrstcr_ptrm_code not in ('28','R3') -- drop weird term parts
#     and sfrstcr_add_date <= '{self.cycle_date}' -- added before cycle_day
#     and (sfrstcr_rsts_date > '{self.cycle_date}' or sfrstcr_rsts_code in ('DC','DL','RD','RE','RW','WD','WF')) -- dropped after cycle_day or still enrolled
#     and ssbsect_subj_code <> 'INST' -- exceptional sections
#     and ssbsect_credit_hrs > 0
# group by
#     {grp}, crse_code
# """

#             qry = f"""
# with CTE as {subqry(qry)}

# --individual courses
# select
#     *
# from
#     CTE

# union all

# --total credit hours
# select
#     {grp}
#     ,'_total_sch' as crse_code
#     ,sum(credit_hr) as credit_hr
# from
#     CTE
# group by
#     {grp}

# union all

# --headcount 
# select
#     {grp}
#     ,'_headcount' as crse_code
#     ,1 as credit_hr
# from
#     CTE
# group by
#     {grp}
# """
#             df = run(qry, show).set_index(['crse_code','pidm'])
#             return df
#         df, new = self.get(fcn, 'registrations')
#         return df



    def get_students(self, show=False):
        def fcn():
            df = (self.admissions
                  .merge(self.get_flags(), on=['pidm','term_code'], how='left', suffixes=['', '_flags'])
                  .merge(self.get_drivetimes(), on=['zip','camp_code'], how='left', suffixes=['', '_zips'])                
            )
            mask = df.eval("drivetime.isnull() & zip.notnull() & camp_code!='o'")
            if mask.any():
                df[mask].set_index(['state','city','zip','camp_code'])[[]].sort_index().reset_index().disp(50)

            for c in ['gap_score','t_gap_score','ftic_gap_score']:
                if c not in df:
                    df[c] = pd.NA
            
            df['gap_score'] = np.where(
                df['styp_code']=='n',
                df['ftic_gap_score'].combine_first(df['t_gap_score']).combine_first(df['gap_score']),
                df['t_gap_score'].combine_first(df['ftic_gap_score']).combine_first(df['gap_score']))
            
            
            # df['oriented'] = np.where(df['orien_sess'].notnull() | df['registered'].notnull(), 'y', np.where(df['orientation_hold_exists'].notnull(), 'n', 'w'))
            df['oriented'] = df['orientation_hold_exists'].isnull() | df['orien_sess'].notnull() | df['registered'].notnull()

            # df['verified'] = np.where(df['ver_complete'].notnull(), 'y', np.where(df['selected_for_ver'].notnull(), 'n', 'w'))
            df['verified'] = df['selected_for_ver'].isnull() | df['ver_complete'].notnull()
            
            df['sat10_total_score'] = (36-9) / (1600-590) * (df['sat10_total_score']-590) + 9
            df['act_equiv'] = df[['act_new_comp_score','sat10_total_score']].max(axis=1)

            df['eager'] = (self.stable_date - df['first_date']).dt.days
            df['age'] = (self.stable_date - df['birth_date']).dt.days

            for k in ['reading', 'writing', 'math']:
                df[f'tsi_{k}'] = ~df[k].isin(['not college ready', 'retest required', pd.NA, None, np.nan])
            
            repl = {'ae':0, 'n1':1, 'n2':2, 'n3':3, 'n4':4, 'r1':1, 'r2':2, 'r3':3, 'r4':4}
            df['hs_qrtl'] = pd.cut(df['hs_pctl'], bins=[-1,25,50,75,90,101], labels=[4,3,2,1,0], right=False).combine_first(df['apdc_code'].map(repl))

            df['lgcy'] = ~df['lgcy_code'].isin(['o',pd.NA,None,np.nan])
            df['resd'] = df['resd_code'] == 'r'

            for k in ['waiver_desc','fafsa_app','ssb_last_accessed','finaid_accepted','schlship_app']:
                df[k.split('_')[0]] = df[k].notnull()



            # df['majr_code'] = df['majr_code'].replace({'0000':pd.NA, 'und':pd.NA, 'eled':'eted', 'agri':'unda'})

            # df['coll_code'] = df['coll_code'].replace({'ae':'an', 'eh':'ed', 'hs':'hl', 'st':'sm', '00':pd.NA})

            # df['coll_desc'] = df['coll_code'].map({
            #     'an': 'ag & natural_resources',
            #     'ba': 'business',
            #     'ed': 'education',
            #     'en': 'engineering',
            #     'hl': 'health sciences',
            #     'la': 'liberal & fine arts',
            #     'sm': 'science & mathematics',
            #     pd.NA: 'no college designated',
            # })



            # checks = [
            #     'cycle_day >= 0',
            #     'eager >= cycle_day',
            #     'age >= 5000',
            #     'distance >= 0',
            #     'hs_pctl >=0',
            #     'hs_pctl <= 100',
            #     'hs_qrtl >= 0',
            #     'hs_qrtl <= 4',
            #     'act_equiv >= 1',
            #     'act_equiv <= 36',
            #     'gap_score >= 0',
            #     'gap_score <= 100',
            # ]
            # for check in checks:
            #     mask = df.eval(check)
            #     assert mask.all(), [check,df[~mask].disp(5)]
            mask = df['cycle_date_flags'].isnull()  # rows from admissions not on flags - should not be any
            if mask.any():
                display(df[mask]['styp_code'].value_counts().sort_index().to_frame().T)
            return df.set_index(['pidm'])
        df, new = self.get(fcn, 'students', [self.get_admissions,self.get_flags,self.get_drivetimes])
        return df
